In [35]:
from dia.model import Dia
import json
import os

In [2]:
model = Dia.from_pretrained("mesolitica/Malaysian-Dia-1.6B", compute_dtype="float16")

/home/husein/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [ ]:
from ctc_forced_aligner import (
    load_audio,
    load_alignment_model,
    generate_emissions,
    preprocess_text,
    get_alignments,
    get_spans,
    postprocess_results,
)
import torch
import torchaudio

device = 'cuda'
alignment_model, alignment_tokenizer = load_alignment_model(
    device,
    dtype=torch.float16 if device == "cuda" else torch.float32,
)

In [4]:
with open('malaymmlu.json') as fopen:
    d = json.load(fopen)

In [189]:
len(d)

20437

In [47]:
d[0]['question'].replace('\n', ' ')

'Pasangan algoritma yang digunakan untuk melakukan penyulitan dan nyahsulit dikenali sebagai A. kunci (keys) B. Sifer (cipher) C. Teks sifer (ciphertext)'

In [154]:
replace = {
    'Aee .': 'A.',
    'Bee .': 'B.',
    'See .': 'C.',
    'Dee .': 'D.',
    'Eee .': 'E.',
    ' .': '.',
    '(': '',
    ')': '',
}

replace_pronunciation = {
    'Aee .': 'Eay .',
    ' .': '.',
    '(': '',
    ')': '',
}

In [183]:
i = 2
speaker = os.path.join('dialects_processed', os.path.split(d[i]['speaker']['audio'])[-1][-50:])
t = d[i]['speaker']['transcription']
clone_from_text = f"[S1] {t}"
clone_from_audio = speaker
t_ = d[i]['pronunciation'].replace('. .', '.')
for k, v in replace.items():
    t_ = t_.replace(k, v)
text = clone_from_text + '[S1] ' + t_.strip()
texts = [text] * 1
clone_from_audios = [clone_from_audio] * len(texts)
text

'[S1] Dalam pada itu, bila menambah dengan kepesatan perkembangan sains dan teknologi ketika ini, teknologi seperti kecerdasan buatan AI dan klon yang tidak wujud pada masa dahulu kini telah ada.[S1] SQL adalah bahasa pengaturcaraan aras tinggi generasi kelima yang ringkas dan mudah. A. Benar. B. Salah.'

In [184]:
output = model.generate(texts, audio_prompt=clone_from_audios, use_torch_compile=True, verbose=True, 
                        max_tokens=3000, temperature = 1.0, cfg_scale = 1.0)

generate: starting generation loop
generate: using use_torch_compile=True, the first step may be slow
generate step 1204: speed=116.171 tokens/s, realtime factor=1.351x
generate step 1290: speed=179.421 tokens/s, realtime factor=2.086x
generate step 1376: speed=180.095 tokens/s, realtime factor=2.094x
generate step 1462: speed=179.728 tokens/s, realtime factor=2.090x
generate step 1548: speed=179.712 tokens/s, realtime factor=2.090x
generate step 1634: speed=179.979 tokens/s, realtime factor=2.093x
generate step 1720: speed=179.801 tokens/s, realtime factor=2.091x
generate: avg steps=635.0, total duration=4.227s


In [185]:
import IPython.display as ipd
ipd.Audio(output, rate = 44100)

In [186]:
new_wav = torch.tensor(output)
audio_waveform = torchaudio.functional.resample(
    new_wav, orig_freq=44100, new_freq=16000
).type(torch.float16).cuda()
emissions, stride = generate_emissions(
    alignment_model, audio_waveform, batch_size=1
)

In [187]:
gen_text = d[i]['pronunciation'].replace('. .', '.')
for k, v in replace_pronunciation.items():
    gen_text = gen_text.replace(k, v)
gen_text

'SQL adalah bahasa pengaturcaraan aras tinggi generasi kelima yang ringkas dan mudah. Eay. Benar. Bee. Salah.'

In [188]:
tokens_starred, text_starred = preprocess_text(
    gen_text,
    romanize=True,
    language='ms',
)
segments, scores, blank_token = get_alignments(
    emissions,
    tokens_starred,
    alignment_tokenizer,
)
spans = get_spans(tokens_starred, segments, blank_token)
word_timestamps = postprocess_results(text_starred, spans, stride, scores)

In [182]:
word_timestamps

[{'start': 0.16, 'end': 0.6, 'text': 'SQL', 'score': -5.690315246582031},
 {'start': 0.72, 'end': 0.96, 'text': 'adalah', 'score': -0.04552757740020752},
 {'start': 1.02, 'end': 1.34, 'text': 'bahasa', 'score': -0.28125274181365967},
 {'start': 1.46,
  'end': 2.12,
  'text': 'pengaturcaraan',
  'score': -4.694266319274902},
 {'start': 2.24, 'end': 2.42, 'text': 'aras', 'score': -0.011003851890563965},
 {'start': 2.5, 'end': 2.74, 'text': 'tinggi', 'score': -0.4039870500564575},
 {'start': 2.86,
  'end': 3.32,
  'text': 'generasi',
  'score': -2.6793646812438965},
 {'start': 3.42, 'end': 3.72, 'text': 'kelima', 'score': -0.09638774394989014},
 {'start': 4.22, 'end': 4.36, 'text': 'yang', 'score': -1.0128624439239502},
 {'start': 4.44, 'end': 4.78, 'text': 'ringkas', 'score': -0.874281644821167},
 {'start': 4.9, 'end': 5.0, 'text': 'dan', 'score': -0.006037473678588867},
 {'start': 5.06, 'end': 5.3, 'text': 'mudah.', 'score': -0.0479506254196167},
 {'start': 5.92, 'end': 6.0, 'text': 'Ea